In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 3060 Laptop GPU


In [2]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset
import random
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor

class ODIRDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the CSV file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.to_tensor = ToTensor()

        # Eliminar columnas redundantes
        self.df = self.df.drop(columns=['labels', 'N', 'D', 'G', 'C', 'A', 'H', 'M', 'O'])

        # Convertir one-hot encoded 'target' a integer class labels
        self.df['target'] = self.df['target'].apply(lambda x: torch.argmax(torch.tensor(eval(x))))
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        
        img_name = os.path.join(self.root_dir, str(self.df.iloc[idx, self.df.columns.get_loc('filename')]))
        image = Image.open(img_name)
        image = self.to_tensor(image)
        
        label = torch.tensor(self.df.iloc[idx]['target'])
        
        if self.transform:
            image = self.transform(image)
            
        return image, label


In [3]:
dataset = ODIRDataset(csv_file='full_df.csv', root_dir='preprocessed_images/preprocessed_images')

print(dataset.df)


        ID  Patient Age Patient Sex    Left-Fundus    Right-Fundus  \
0        0           69      Female     0_left.jpg     0_right.jpg   
1        1           57        Male     1_left.jpg     1_right.jpg   
2        2           42        Male     2_left.jpg     2_right.jpg   
3        4           53        Male     4_left.jpg     4_right.jpg   
4        5           50      Female     5_left.jpg     5_right.jpg   
...    ...          ...         ...            ...             ...   
6387  4686           63        Male  4686_left.jpg  4686_right.jpg   
6388  4688           42        Male  4688_left.jpg  4688_right.jpg   
6389  4689           54        Male  4689_left.jpg  4689_right.jpg   
6390  4690           57        Male  4690_left.jpg  4690_right.jpg   
6391  4784           58        Male  4784_left.jpg  4784_right.jpg   

                               Left-Diagnostic Keywords  \
0                                              cataract   
1                                        

/tmp/ipykernel_6118/596204278.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.df.iloc[idx]['target'])


In [4]:
from torch.utils.data import DataLoader, random_split


train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=4)

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

num_classes = 8

model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

model.fc = nn.Linear(model.fc.in_features, num_classes)

torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print(model)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

from torch.cuda.amp import autocast, GradScaler

# Training loop
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    scaler = GradScaler()
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        
        optimizer.zero_grad()

        with autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        #loss.backward()
        #optimizer.step()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        running_loss += loss.item() * images.size(0)
        
       
        _, predicted = torch.max(outputs, 1)
      
    
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
       
    
    epoch_loss = running_loss / total
    epoch_accuracy = correct / total
    
    return epoch_loss, epoch_accuracy

In [7]:
def validate_model(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

  
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            
            _, predicted = torch.max(outputs, 1)
            
    
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
    
    epoch_loss = running_loss / total
    epoch_accuracy = correct / total
    
    return epoch_loss, epoch_accuracy

In [8]:
num_epochs = 10

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

for epoch in range(num_epochs):
    train_loss, train_accuracy = train_model(model, train_loader, criterion, optimizer, device)
    val_loss, val_accuracy = validate_model(model, test_loader, criterion, device)
    
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    
    print(f'Epoch {epoch+1}/{num_epochs}')
    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
    print(f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

torch.save(model.state_dict(), 'resnet50_odir.pth')


/tmp/ipykernel_6118/596204278.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.df.iloc[idx]['target'])
/tmp/ipykernel_6118/596204278.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.df.iloc[idx]['target'])
/tmp/ipykernel_6118/596204278.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.df.iloc[idx]['target'])
/tmp/ipykernel_6118/596204278.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sour

Epoch 1/10
Train Loss: 1.4342, Train Accuracy: 0.4659
Val Loss: 1.3460, Val Accuracy: 0.5137


/tmp/ipykernel_6118/596204278.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.df.iloc[idx]['target'])
/tmp/ipykernel_6118/596204278.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.df.iloc[idx]['target'])
/tmp/ipykernel_6118/596204278.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.df.iloc[idx]['target'])
/tmp/ipykernel_6118/596204278.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sour

Epoch 2/10
Train Loss: 1.2954, Train Accuracy: 0.5148
Val Loss: 1.2938, Val Accuracy: 0.5270


/tmp/ipykernel_6118/596204278.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.df.iloc[idx]['target'])
/tmp/ipykernel_6118/596204278.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.df.iloc[idx]['target'])
/tmp/ipykernel_6118/596204278.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.df.iloc[idx]['target'])
/tmp/ipykernel_6118/596204278.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sour

...

Epoch 8/10
Train Loss: 1.1394, Train Accuracy: 0.5918
Val Loss: 1.1096, Val Accuracy: 0.6325


/tmp/ipykernel_6118/596204278.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.df.iloc[idx]['target'])
/tmp/ipykernel_6118/596204278.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.df.iloc[idx]['target'])
/tmp/ipykernel_6118/596204278.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.df.iloc[idx]['target'])
/tmp/ipykernel_6118/596204278.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sour

Epoch 9/10
Train Loss: 1.0797, Train Accuracy: 0.6171
Val Loss: 1.1229, Val Accuracy: 0.6325


/tmp/ipykernel_6118/596204278.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.df.iloc[idx]['target'])
/tmp/ipykernel_6118/596204278.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.df.iloc[idx]['target'])
/tmp/ipykernel_6118/596204278.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.df.iloc[idx]['target'])
/tmp/ipykernel_6118/596204278.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sour

Epoch 10/10
Train Loss: 1.0480, Train Accuracy: 0.6272
Val Loss: 1.2495, Val Accuracy: 0.5887
